# Adverserial Variational Optimization

*Gilles Louppe & Kayle Cranmer*

Notebook by Joeri Hermans

In [1]:
!date

Tue Oct 10 16:07:18 CEST 2017


Complex computer simulators are increasingly used across fields of science as generative models tying parameters of an underlying theory to experimental observations. Inference in this setup is often difficult, as simulators rarely admit a tractable density or likelihood function. We introduce Adversarial Variational Optimization (AVO), a likelihood-free inference algorithm for fitting a non-differentiable generative model incorporating ideas from empirical Bayes and variational inference. We adapt the training procedure of generative adversarial networks by replacing the differentiable generative network with a domain-specific simulator. We solve the resulting non-differentiable minimax problem by minimizing variational upper bounds of the two adversarial objectives. Effectively, the procedure results in learning a proposal distribution over simulator parameters, such that the corresponding marginal distribution of the generated data matches the observations. We present results of the method with simulators producing both discrete and continuous data.

**Manuscript**: [https://arxiv.org/abs/1707.07113](https://arxiv.org/abs/1707.07113)

## Configuration

### Imports

In [2]:
import numpy as np
import torch

from sklearn.utils import check_random_state

### Globals

In [3]:
seed = 1337
random_number_generator = check_random_state(seed)
batch_size = 64
num_epochs = 300
lambda_gp = 0.0025
theta_true = np.array([(42.0-40)/(50-40), (0.9 - 0.5) / (1.5-0.5)])

## Introduction

TODO Explain goals.

## Simulator

Definition of the Physics simulator. TODO Add additional information and explenation of the simulator.

In [8]:
def a_fb(sqrtshalf, gf):
    mz = 90
    gf_nom = 1.0
    sqrts = sqrtshalf * 2.
    a_fb_en = np.tanh((sqrts - mz) / mz * 10)
    a_fb_gf = gf / gf_nom

    return 2 * a_fb_en * a_fb_gf

In [5]:
def diffxsec(costheta, sqrtshalf, gf):
    norm = 2. * (1. + 1. / 3.)
    return ((1 + costheta ** 2) + a_fb(sqrtshalf, gf) * costheta) / norm

In [6]:
def rej_sample_costheta(n_samples, theta, rng):
    sqrtshalf = theta[0] * (50 - 40) + 40
    gf = theta[1] * (1.5 - 0.5) + 0.5

    ntrials = 0
    samples = []
    x = np.linspace(-1, 1, num=1000)
    maxval = np.max(diffxsec(x, sqrtshalf, gf))

    while len(samples) < n_samples:
        ntrials = ntrials + 1
        xprop = rng.uniform(-1, 1)
        ycut = rng.rand()
        yprop = diffxsec(xprop, sqrtshalf, gf) / maxval
        if (yprop / maxval) < ycut:
            continue
        samples.append(xprop)

    return np.array(samples)

In [7]:
def simulator(theta, n_samples, random_state=None):
    rng = check_random_state(random_state)
    samples = rej_sample_costheta(n_samples, theta, rng)

    return samples.reshape(-1, 1)

## Test

In [13]:
X_obs = simulator(theta_true, 50000, random_state=random_number_generator)
num_params = len(theta_true)
num_features = X_obs.shape[1]

In [14]:
print("Number of parameters: " + str(num_params))
print("Number of features: " + str(num_features))

Number of parameters: 2
Number of features: 1


## Conclusion